In [46]:
import sys
sys.path.append("../../current_work/")
from pandas.core.indexing import maybe_convert_ix
import PredictionManager as pm
import backtest_class as backTest
import yFinance_class as yfHelper

## Data'nın indirilmesi

In [48]:
# use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        #period = "ytd",

# fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        #interval = "1m",

In [49]:
userInput = "SASA.IS"
period = '1y'
interval = '1d'

In [50]:
yData = yfHelper.yFinance(userInput.upper(), period, interval).getData()

[*********************100%***********************]  1 of 1 completed


In [51]:
yData.iloc[0]

Date         2020-12-23 00:00:00
Open                   14.302684
High                    14.56947
Low                    14.280452
Close                  14.436077
Adj Close              14.436077
Volume                  10815611
Name: 0, dtype: object

In [52]:
yData.iloc[-1]

Date         2021-12-23 00:00:00
Open                        45.0
High                   45.560001
Low                    40.740002
Close                  43.279999
Adj Close              43.279999
Volume                  28815358
Name: 249, dtype: object

In [55]:
df = yData.copy()
df = df.set_index(pd.DatetimeIndex(df['Date'].values))

In [56]:
col_list = df.columns.to_list()

In [57]:
col_list

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

In [60]:
len(col_list)

7

In [61]:
col_list[1:len(col_list)-1]

['Open', 'High', 'Low', 'Close', 'Adj Close']

In [64]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
2020-12-23,2020-12-23,14.302684,14.569470,14.280452,14.436077,14.436077,10815611
2020-12-24,2020-12-24,14.465720,14.650988,14.361970,14.443488,14.443488,7477201
2020-12-25,2020-12-25,14.525006,14.710274,14.436077,14.458309,14.458309,8087790
2020-12-28,2020-12-28,14.525006,14.673220,14.421256,14.443488,14.443488,5901864
2020-12-29,2020-12-29,14.458309,15.858935,14.384202,14.865899,14.865899,57816868
...,...,...,...,...,...,...,...
2021-12-17,2021-12-17,58.400002,61.000000,51.250000,52.000000,52.000000,47415546
2021-12-20,2021-12-20,49.980000,54.650002,47.200001,51.549999,51.549999,35937805
2021-12-21,2021-12-21,52.099998,54.000000,46.400002,46.599998,46.599998,28637408
2021-12-22,2021-12-22,45.400002,47.860001,43.599998,43.980000,43.980000,37211464


In [67]:
type(df.iloc[2,1])

numpy.float64

In [69]:
for i in range(0,10):
    if (df.iloc[i,2] > 0):
        print("ok")
        print(df.iloc[i,2])

ok
14.569470405578613
ok
14.65098762512207
ok
14.710273742675781
ok
14.673219680786133
ok
15.858935356140137
ok
15.280899047851562
ok
15.117862701416016
ok
15.458756446838379
ok
15.458756446838379
ok
15.725542068481445


## RSI parametresinin çıkartılması

In [7]:
import pandas as pd
import numpy as np

In [30]:
close = yData.Close
ret = close.diff() #(n+1).gün - n.gün
up = []
down = [] 

for element in range(0, len(ret)):
    if (ret[element] < float(0)):
        up.append(0)
        down.append(ret[element])
    elif (ret[element] >= float(0)):
        up.append(ret[element])
        down.append(0)
        
upSeries = pd.Series(up)
downSeries = pd.Series(down).abs()

In [32]:
ret = ret.replace(np.nan, 0)
ret

0      0.000000
1      0.163035
2     -0.281608
3      0.244554
4      0.207500
         ...   
245    1.560001
246    4.269997
247   -1.000000
248    5.050003
249   -4.900002
Name: Close, Length: 250, dtype: float64

In [10]:
lookback = 7
upEwm = upSeries.ewm(com = lookback -1, adjust = False).mean()
downEwm = downSeries.ewm(com = lookback -1, adjust = False).mean()

In [11]:
rs = upEwm / downEwm
rsi = 100 - (100 / (1 + rs))

In [12]:
close = close.drop(index = close.index[0])
temp_df = pd.DataFrame(rsi).rename(columns={0 : 'rsi'}).set_index(close.index)


In [13]:
print("toplam gün sayısı:", len(yData))
print("rsi veri gün sayısı:", len(rsi))

toplam gün sayısı: 250
rsi veri gün sayısı: 249


In [14]:
#bu işlemi yapmamızın sebebi hesaplamalarda ilk iki gün 
#RSI değerinin 100 çıkması
#o yüzden ilk 2 günü pass geçiyoruz hatta 3 günü de geçebiliriz.
yData["RSI_14"] = temp_df[2:]

## Algoritmanın geliştirilmesi

### 1- RSI ile alım satım

In [15]:
lower_band = np.float(30)
upper_band = np.float(70)
signal = 0 #sinyal parametresi sattıysak -1 aldıysak 1 olacak.
rsi_signal = []
buy_price = [] #alım yaptıgımızda alım fiyatını ekledigimiz liste
sell_price = []#satış yaptıgımızda satış fiyatını ekledigimiz liste
rsi = yData.RSI_14
prices = yData.Close
def wait_for_signal(buyList, sellList, signalList):
    buyList.append(np.nan)
    sellList.append(np.nan)
    signalList.append(0)
    
def fill_buy_signal(buyList,buyPrice, sellList, signalList):
    buyList.append(buyPrice)
    sellList.append(np.nan)
    signalList.append(1)
    
def fill_sell_signal(buyList, sellList, sellPrice, signalList):
    buyList.append(np.nan)
    sellList.append(sellPrice)
    signalList.append(-1)
    
for element in range(1, len(rsi)):
    if ((rsi[element - 1] > lower_band) and (rsi[element] < lower_band)):
        #eğer rsi değeri 30'un üstünden 30'un altına geçiş yaptıysa
        #alım yapmak istiyoruz
        #ancak iki kere alım yapamayacağımızdan sinyal kontrolü yap.
        if (signal != 1):
            fill_buy_signal(buy_price, prices[element], sell_price, rsi_signal)
            signal = 1
        else:
            wait_for_signal(buy_price, sell_price, rsi_signal)
            
    elif((rsi[element - 1] < upper_band) and (rsi[element] > upper_band)):
        #eğer rsi değeri 70'in altındaysa ve 70'in üstüne geçiş yaptıysa
        #satış yapmak istiyoruz
        #ancak iki kere satış yapamacağımızdan sinyal kontrolü yap
        if (signal != -1):
            fill_sell_signal(buy_price, sell_price, prices[element], rsi_signal)
            signal = -1
        else:
            wait_for_signal(buy_price, sell_price, rsi_signal)
            
    else:
        wait_for_signal(buy_price, sell_price, rsi_signal)
            
            
        

/Users/batuhanduyuler/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/Users/batuhanduyuler/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [16]:
investmenValue = yData['Close'][0] * 5
firstInvestment = investmenValue
decisionFlag = False
stockNumber = round(investmenValue / yData['Close'][0])


for i in range(0, len(rsi) - 1):
    if (rsi_signal[i] != 0):
        if (decisionFlag == False):
            if(rsi_signal[i] == 1):
                decisionFlag = True
            elif(rsi_signal[i] == -1):
                decisionFlag = True
                investmenValue = yData["Close"][i+1] * stockNumber    
        else:
            if (rsi_signal[i] == 1):
                stockNumber = round(investmenValue / yData['Close'][i+1])
                investmenValue = 0
            elif (rsi_signal[i] == -1):
                investmenValue = yData['Close'][i+1] * stockNumber

                stockNumber = 0
      


if (investmenValue == 0.0):
    investmenValue = yData['Close'][len(yData) - 1] * stockNumber
    stockNumber = 0
    
print("% ",(investmenValue - firstInvestment) / firstInvestment * 100)
print(investmenValue - firstInvestment)

%  18.276098496562298
12.8870210647583


In [17]:
print("işlem yapılmasa durum: %",(yData.Close.iloc[-1] - yData.Close.iloc[0]) / yData.Close.iloc[0]*100)

işlem yapılmasa durum: % 268.72645627163166


In [18]:
alim_satim = 0
for i in rsi_signal:
    alim_satim += abs(i)
    
print("alım satım sayısı: ", alim_satim)
print("veri uzunlugu: ", len(yData))

alım satım sayısı:  3
veri uzunlugu:  250


### 2- Safe Sell RSI algoritması

In [19]:
lower_band = np.float(30)
upper_band = np.float(70)
signal = 0 #sinyal parametresi sattıysak -1 aldıysak 1 olacak.
rsi_signal = []
buy_price = [] #alım yaptıgımızda alım fiyatını ekledigimiz liste
sell_price = []#satış yaptıgımızda satış fiyatını ekledigimiz liste
rsi = yData.RSI_14
beklenenKar = 0.1
prices = yData.Close
lastBuyPrice = []
def wait_for_signal(buyList, sellList, signalList):
    buyList.append(np.nan)
    sellList.append(np.nan)
    signalList.append(0)
    
def fill_buy_signal(buyList,buyPrice, sellList, signalList):
    buyList.append(buyPrice)
    sellList.append(np.nan)
    signalList.append(1)
    
def fill_sell_signal(buyList, sellList, sellPrice, signalList):
    buyList.append(np.nan)
    sellList.append(sellPrice)
    signalList.append(-1)
    
for element in range(1, len(rsi)):
    if ((rsi[element - 1] > lower_band) and (rsi[element] < lower_band)):
        #eğer rsi değeri 30'un üstünden 30'un altına geçiş yaptıysa
        #alım yapmak istiyoruz
        #ancak iki kere alım yapamayacağımızdan sinyal kontrolü yap.
        if (signal != 1):
            fill_buy_signal(buy_price, prices[element], sell_price, rsi_signal)
            signal = 1
            lastBuyPrice.append(prices[element])
        else:
            #RSI alım sinyali verdi ama zaten alım yapmışız
            #fakat karımız %5 üzerindeyse satış yapmak istiyoruz.
            #o yüzden ilk önce fiyat düşüşte mi ona bakıyoruz
            if (prices[element - 1] > prices[element]):
                #eğer düşüşteyse aldığımız fiyattan yüksek mi
                #onu kontrol etmeliyiz.
                if (prices[element] > lastBuyPrice[-1]):
                    #eğer öyleyse karımızı hesaplamalıyız
                    lossPortion = (prices[element] - lastBuyPrice[-1]) / lastBuyPrice[-1]
                    if (lossPortion > beklenenKar ):
                        print(lossPortion)
                        fill_sell_signal(buy_price, sell_price, prices[element], rsi_signal)
                        signal = -1
                    else:
                        wait_for_signal(buy_price, sell_price, rsi_signal)
                else:
                    wait_for_signal(buy_price, sell_price, rsi_signal)
            else:
                wait_for_signal(buy_price, sell_price, rsi_signal)
            
    elif((rsi[element - 1] < upper_band) and (rsi[element] > upper_band)):
        #eğer rsi değeri 70'in altındaysa ve 70'in üstüne geçiş yaptıysa
        #satış yapmak istiyoruz
        #ancak iki kere satış yapamacağımızdan sinyal kontrolü yap
        if (signal != -1):
            fill_sell_signal(buy_price, sell_price, prices[element], rsi_signal)
            signal = -1
        else:
            wait_for_signal(buy_price, sell_price, rsi_signal)
         #satış pozisyonunda bir daha satış yapamayacağımızdan burayı pass geçiyoruz.   
    else:
        #eğer bu durumlardan biri yaşanmadıysa, karı kontrol edebiliriz.
        #ilk olarak sinyal alım pozisyonunda mı ona bakıyoruz.
        if (signal == 1):
            if (prices[element - 1] > prices[element]):
                #fiyat düşüştüyse tespit ettik.
                if (prices[element] > lastBuyPrice[-1]):
                    #karda olup olmadığımızı kontrol ettik
                    lossPortion = (prices[element] - lastBuyPrice[-1]) / lastBuyPrice[-1]
                    if (lossPortion > beklenenKar):
                        print(lossPortion)
                        fill_sell_signal(buy_price, sell_price, prices[element], rsi_signal)
                        signal = -1
                    else:
                        wait_for_signal(buy_price, sell_price, rsi_signal) 
                else:
                    wait_for_signal(buy_price, sell_price, rsi_signal)
            else:
                wait_for_signal(buy_price, sell_price, rsi_signal)
        else:
            wait_for_signal(buy_price, sell_price, rsi_signal)
            
        

/Users/batuhanduyuler/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/Users/batuhanduyuler/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [20]:
alim_satim = 0
for i in rsi_signal:
    alim_satim += abs(i)
    
print("alım satım sayısı: ", alim_satim)
print("veri uzunlugu: ", len(yData))

alım satım sayısı:  3
veri uzunlugu:  250


In [72]:
len(rsi_signal)

249

In [74]:
new_data = yData.to_string(index = False)

In [22]:
print("% ",(investmenValue - firstInvestment) / firstInvestment * 100)
print(investmenValue - firstInvestment)

%  18.276098496562298
12.8870210647583


In [23]:
print("işlem yapılmasa durum: %",(yData.Close.iloc[-1] - yData.Close.iloc[0]) / yData.Close.iloc[0] * 100)

işlem yapılmasa durum: % 268.72645627163166


In [24]:
yData

,Date,Open,High,Low,Close,Adj Close,Volume,RSI_14
0,2020-12-17,14.161881,14.584292,14.021077,14.102595,14.102595,15330790,NaN
1,2020-12-18,14.028488,14.502774,14.028488,14.265631,14.265631,17246022,NaN
2,2020-12-21,14.043309,14.302684,13.887684,13.984023,13.984023,14780192,NaN
3,2020-12-22,14.035898,14.421256,14.035898,14.228577,14.228577,10151210,81.774532
4,2020-12-23,14.302684,14.569470,14.280452,14.436077,14.436077,10815611,84.591091
...,...,...,...,...,...,...,...,...
245,2021-12-13,47.180000,49.340000,47.180000,48.580002,48.580002,22769160,76.616154
246,2021-12-14,49.139999,52.849998,49.139999,52.849998,52.849998,30638421,85.314248
247,2021-12-15,53.599998,54.799999,51.500000,51.849998,51.849998,28540846,77.443560
248,2021-12-16,52.900002,57.000000,52.700001,56.900002,56.900002,41337525,85.386530


In [44]:
temp_df = pd.DataFrame(list(zip(yData.Date[1:].to_list(), yData.Close[1:].to_list(), rsi_signal)), columns = ["Date", "Close", "Signal"])

In [45]:
temp_df.Signal.unique()

array([ 0, -1,  1])